<a href="https://colab.research.google.com/github/IgnacioMurubeCrego/practica5ArqCompu/blob/main/practica5_ArqCompu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sl12sro2
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sl12sro2
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-wt61d08c/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpl9xwgq25".


In [ ]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello World from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 3>>>();
    cudaDeviceSynchronize();
}

Hello World from block: 0, thread: 0
Hello World from block: 0, thread: 1
Hello World from block: 0, thread: 2
Hello World from block: 1, thread: 0
Hello World from block: 1, thread: 1
Hello World from block: 1, thread: 2



### Ejemplo B :  duplicar un valor con CUDA


In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

// DEFINICIÓN KERNEL CUDA

__global__ void duplicarValor(int *valor_device)

{
  printf("Block [2%u], Thread [2%u]: valor_device=%d\n", blockIdx.x, threadIdx.x, *valor_device);
  *valor_device *= 2;
}

// FUNCIÓN DEL HOST

int main()
{

  // 1 - INICIALIZAR DATOS DEL HOST

  int valor_host = 5;

  // valor inicial en el host

  // 2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *valor_device; // puntero al device

  cudaMalloc((void **)&valor_device, sizeof(int)); // asignar memoria para el device

  // 3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(valor_device, &valor_host, sizeof(int), cudaMemcpyHostToDevice); // transferir el valor del host al device

  // 4 - EJECUTAR EN UNO O MÁS KERNELS

  duplicarValor<<<1, 2>>>(valor_device); // lanzar el kernel

  cudaDeviceSynchronize(); // espera hasta que el kernel termine

  // 5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(&valor_host, valor_device, sizeof(int), cudaMemcpyDeviceToHost); // transferir el valor del device al host

  // 6 - LIBERAR RECURSOS

  cudaFree(valor_device); // liberar memoria del device

  // MOSTRAR RESULTADO

  std::cout << "Resultado: " << valor_host << std::endl;

  return 0;
}

Block [20], Thread [20]: valor_device=5
Block [20], Thread [21]: valor_device=5
Resultado: 10



### Ejemplo C : suma de vectores con CUDA


In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 100

//DEFINICIÓN KERNEL CUDA

__global__ void sumaVectores(int* v1_device, int* v2_device, int*suma_device) {
  printf("Índice de hilo dentro del bloque: %d\n", threadIdx.x);
  printf("Índice del bloque: %d\n", blockIdx.x);
  printf("Tamaño del bloque (número de hilos): %d\n", blockDim.x);
  for (int i = 0; i < T_VEC; i++) {
    suma_device[i] = v1_device[i] + v2_device[i];
  }
}

//FUNCIÓN DEL HOST

int main() {

  //1 - INICIALIZAR DATOS DEL HOST

  int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
  for (int i = 0; i < T_VEC; i++) {
    v1_host[i] = 1;
    v2_host[i] = 2;
  }

  //2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *v1_device, *v2_device, *suma_device; //punteros al device
  cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
  cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
  cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);

  //3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //transferir el valor del host al device
  cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice);
  //cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //Inutil

  //4 - EJECUTAR EN UNO O MÁS KERNELS

  sumaVectores<<<1, 1>>>(v1_device, v2_device, suma_device);
  //lanzar el kernel
  cudaDeviceSynchronize(); //espera hasta que el kernel termine

  //5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
  cudaMemcpyDeviceToHost); //transferir el valor del device al host

  //6 - LIBERAR RECURSOS
  cudaFree(v1_device); //liberar memoria del device
  cudaFree(v2_device);
  cudaFree(suma_device);

  //MOSTRAR RESULTADO
  for (int i = 0; i < T_VEC; i++) {
    std::cout << suma_host[i] << " ";
  }
  std::cout << "\n";
 return 0;
}

Índice de hilo dentro del bloque: 0
Índice del bloque: 0
Tamaño del bloque (número de hilos): 1
3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 



#Ejercicio 1 (4 puntos)
Modificar el ejemplo de suma de vectores con CUDA para que se ejecute en un bloque de
25 hilos, dónde cada uno se encargue de sumar una parte de los vectores.

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
#define T_VEC 100

//DEFINICIÓN KERNEL CUDA

__global__ void sumaVectores(int* v1_device, int* v2_device, int*suma_device) {
  // printf("Índice de hilo dentro del bloque: %d\n", threadIdx.x);
  // printf("Índice del bloque: %d\n", blockIdx.x);
  // printf("Tamaño del bloque (número de hilos): %d\n", blockDim.x);
  int threadInBlockIndex = threadIdx.x + blockIdx.x * blockDim.x;

  // Cada hilo suma un cuarto del vector
    for (int i = threadInBlockIndex; i < T_VEC; i += blockDim.x * gridDim.x) {
        suma_device[i] = v1_device[i] + v2_device[i];
    }
}

//FUNCIÓN DEL HOST

int main() {

  //1 - INICIALIZAR DATOS DEL HOST

  int v1_host[T_VEC], v2_host[T_VEC], suma_host[T_VEC];
  for (int i = 0; i < T_VEC; i++) {
    v1_host[i] = 1;
    v2_host[i] = 2;
  }

  //2 - DECLARAR Y ASIGNAR MEMORIA PARA EL DEVICE

  int *v1_device, *v2_device, *suma_device; //punteros al device
  cudaMalloc((void**)&v1_device, sizeof(int) * T_VEC); //asignar memoria para el device
  cudaMalloc((void**)&v2_device, sizeof(int) * T_VEC);
  cudaMalloc((void**)&suma_device, sizeof(int) * T_VEC);

  //3 - TRANSFERIR DATOS DEL HOST AL DEVICE

  cudaMemcpy(v1_device, v1_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //transferir el valor del host al device
  cudaMemcpy(v2_device, v2_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice);
  //cudaMemcpy(suma_device, suma_host, sizeof(int) * T_VEC,cudaMemcpyHostToDevice); //Inutil

  //4 - EJECUTAR EN UNO O MÁS KERNELS

  sumaVectores<<<1, 25>>>(v1_device, v2_device, suma_device);
  //lanzar el kernel
  cudaDeviceSynchronize(); //espera hasta que el kernel termine

  //5 - TRANSFERIR DATOS DEL DEVICE AL HOST

  cudaMemcpy(suma_host, suma_device, sizeof(int) * T_VEC,
  cudaMemcpyDeviceToHost); //transferir el valor del device al host

  //6 - LIBERAR RECURSOS
  cudaFree(v1_device); //liberar memoria del device
  cudaFree(v2_device);
  cudaFree(suma_device);

  //MOSTRAR RESULTADO
  for (int i = 0; i < T_VEC; i++) {
    std::cout << suma_host[i] << " ";
  }
  std::cout << "\n";
 return 0;
}

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 



#Ejercicio 2 (5 puntos)
Desarrollar un programa donde se realice la suma de cinco vectores de 200 valores con
CUDA de manera paralela. Se debe ejecutar con 10 bloques de 20 hilos.

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>
#define VECTOR_SIZE 200
#define THREADS_PER_BLOCK 20
#define BLOCKS 10

// Kernel para sumar cinco vectores
__global__ void sumaVectores(int* v1_device, int* v2_device, int* v3_device, int* v4_device, int* v5_device, int* suma_device) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x; // Índice global del hilo

    if (tid < VECTOR_SIZE) { // Verificar que el hilo está dentro del rango válido
        suma_device[tid] = v1_device[tid] + v2_device[tid] + v3_device[tid] + v4_device[tid] + v5_device[tid];
    }
}

int main() {
    // 1. Inicializar los vectores en el host
    int v1_host[VECTOR_SIZE], v2_host[VECTOR_SIZE], v3_host[VECTOR_SIZE], v4_host[VECTOR_SIZE], v5_host[VECTOR_SIZE], suma_host[VECTOR_SIZE];

    for (int i = 0; i < VECTOR_SIZE; i++) {
        v1_host[i] = 1;
        v2_host[i] = 2;
        v3_host[i] = 3;
        v4_host[i] = 4;
        v5_host[i] = 5;
    }

    // 2. Declarar y asignar memoria para el device
    int *v1_device, *v2_device, *v3_device, *v4_device, *v5_device, *suma_device;
    cudaMalloc((void**)&v1_device, sizeof(int) * VECTOR_SIZE);
    cudaMalloc((void**)&v2_device, sizeof(int) * VECTOR_SIZE);
    cudaMalloc((void**)&v3_device, sizeof(int) * VECTOR_SIZE);
    cudaMalloc((void**)&v4_device, sizeof(int) * VECTOR_SIZE);
    cudaMalloc((void**)&v5_device, sizeof(int) * VECTOR_SIZE);
    cudaMalloc((void**)&suma_device, sizeof(int) * VECTOR_SIZE);

    // 3. Transferir datos del host al device
    cudaMemcpy(v1_device, v1_host, sizeof(int) * VECTOR_SIZE, cudaMemcpyHostToDevice);
    cudaMemcpy(v2_device, v2_host, sizeof(int) * VECTOR_SIZE, cudaMemcpyHostToDevice);
    cudaMemcpy(v3_device, v3_host, sizeof(int) * VECTOR_SIZE, cudaMemcpyHostToDevice);
    cudaMemcpy(v4_device, v4_host, sizeof(int) * VECTOR_SIZE, cudaMemcpyHostToDevice);
    cudaMemcpy(v5_device, v5_host, sizeof(int) * VECTOR_SIZE, cudaMemcpyHostToDevice);

    // 4. Ejecutar el kernel
    sumaVectores<<<BLOCKS, THREADS_PER_BLOCK>>>(v1_device, v2_device, v3_device, v4_device, v5_device, suma_device);
    cudaDeviceSynchronize(); // Esperar a que termine el kernel

    // 5. Transferir los resultados de vuelta al host
    cudaMemcpy(suma_host, suma_device, sizeof(int) * VECTOR_SIZE, cudaMemcpyDeviceToHost);

    // 6. Liberar memoria del device
    cudaFree(v1_device);
    cudaFree(v2_device);
    cudaFree(v3_device);
    cudaFree(v4_device);
    cudaFree(v5_device);
    cudaFree(suma_device);

    // 7. Mostrar los resultados
    std::cout << "Resultado de la suma de los cinco vectores:" << std::endl;
    for (int i = 0; i < VECTOR_SIZE; i++) {
        std::cout << suma_host[i] << " ";
    }
    std::cout << "\n";

    return 0;
}


Resultado de la suma de los cinco vectores:
15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 



# Ejercicio 3 (1 puntos)
###Explicar en qué casos se usarían más o menos número de bloques, hilos por bloque, etc.
### Por ejemplo, cuando interesa utilizar solo un bloque con muchos hilos, muchos bloques con pocos hilos, o un punto intermedio.


La elección entre usar más bloques, más hilos por bloque o un punto intermedio entre ambas depende de varios factores, como la arquitectura de la GPU, el tamaño del problema, y el tipo de operación que se quiere aplicar.

En general, optar por utilizar un bloque con muchos hilos es preferible para procesar cantidades pequeñas de datos, especialmente cuando los datos están relacioados entre sí, o deben ser compartidos con el resto de hilos. Es decir, cuando la comunicación entre hilos es necesaria un bloque con muchos hilos favorece la facilidad de sincronización y acceso entre hilos.

Luego, en el caso de trabajar con grandes cantidades de datos es conveniente optar por un mayor número de bloques con menos hilos dado que esto facilita la escalabilidad para resolver problemas grandes y permite distribuir el trabajo en múltiples bloques donde se aprovecha mejor el paralelismo masivo de la GPU.